In [32]:
from multiprocessing import Pool, Barrier
import numpy as np
import math
import time
from tabulate import tabulate

In [2]:
def butterfl_op(arr, block_per_process, extra, ln, wlen_pw):
    max_deg = ln >> 1
    for k in range(block_per_process + extra):
        for j in range(max_deg):
            u, v = arr[k*ln + j], arr[k*ln + max_deg + j] * wlen_pw[j]
            arr[k*ln + j] = u + v
            arr[k*ln + max_deg + j] = u - v
            
    return arr

In [3]:
def fft(arr, inv, n, num_pr):
    if (n == 1):
        return
    
    j = 0
    for i in range(1, n):
        bit = n >> 1
        while (j & bit):
            j ^= bit 
            bit >>= 1
            
        j ^= bit
        if (i < j):
            arr[i], arr[j] = arr[j], arr[i]

    wlen_pw = np.zeros(n, dtype=np.complex128)

    if n >= 4:
        conj = 1j* (-1 if inv else 1)
        for i in range(0, n, 4):
            t1, t2 = arr[i], arr[i+1]
            t3 = arr[i+2]
            arr[i] = t1 + t2 + t3 + arr[i + 3]
            arr[i + 1] = t1 - t2 - conj*(t3 - arr[i + 3])
            arr[i + 2] = t1 + t2 - t3 - arr[i + 3]
            arr[i + 3] = t1 - t2 + conj*(t3 - arr[i + 3])
    elif (n >= 2):
        for i in range(0, n, 2):
            arr[i], arr[i + 1] = arr[i] + arr[i + 1], arr[i] - arr[i + 1]
        
    ln = 8
    while (ln <= n):
        max_deg = ln >> 1
        ang = 2 * np.pi / ln * (1 if inv else -1)
        wlen = np.cos(ang) + 1j*np.sin(ang)
        wlen_pw = np.power(wlen, np.arange(0, max_deg), dtype=np.complex128)
        
        n_blocks = n // ln
        if n_blocks < num_pr:
            num_pr = n_blocks
        
        block_per_process = n_blocks // num_pr
        rem_blocks = n_blocks % num_pr
        extra = [0] * num_pr
        offset = [0] * num_pr
        
        ans = [0] * num_pr
        pool = Pool(num_pr)
        
        
        for i in range(num_pr):
            offset[i] = i * block_per_process + min(i, rem_blocks)
            extra[i] = 1 if (i < rem_blocks) else 0
            ans[i] = pool.apply_async(func=butterfl_op, args=(arr[offset[i]*ln: offset[i]*ln + (block_per_process + extra[i])*ln], block_per_process, extra[i], ln, wlen_pw))
        
        
        [ans[i].wait() for i in range(num_pr)]
        arr = np.concatenate([ans[i].get() for i in range(num_pr)])
        
        
        ln <<= 1
 

    if (inv):
        arr /= n   
    
    return arr

In [9]:
num_thr = [1, 2, 3, 4, 5]
n = [1000, 10000, 100000, 1000000]
time_ex, err = [], []

In [10]:
for n_iter in n:
    for n_thr in num_thr:
        a = np.array([i for i in range(n_iter)], dtype=complex)
        two_degr = pow(2, math.ceil(math.log2(len(a))))
        a = np.concatenate([a, np.zeros(two_degr - len(a))])
        
        #start_time1 = time.time()
        ans1 = np.fft.ifft(a, n=two_degr)
        #end_time1 = time.time()

        start_time2 = time.time()
        ans2 = fft(a, True, two_degr, n_thr)
        end_time2 = time.time()

        time_ex.append(end_time2-start_time2)
        err.append(np.linalg.norm(ans1- ans2))

In [35]:
timex_ex_np = np.array(time_ex).reshape(4, -1)
table = tabulate(np.hstack([np.array(n).reshape(4, 1), timex_ex_np]), num_thr, tablefmt="fancy_grid")

print(table)

╒════════════╤═══════════╤══════════╤══════════╤══════════╤══════════╕
│            │         1 │        2 │        3 │        4 │        5 │
╞════════════╪═══════════╪══════════╪══════════╪══════════╪══════════╡
│   1000     │  0.11123  │ 0.139001 │ 0.169494 │ 0.188572 │ 0.227099 │
├────────────┼───────────┼──────────┼──────────┼──────────┼──────────┤
│  10000     │  0.349064 │ 0.399931 │ 0.387643 │ 0.473554 │ 0.497944 │
├────────────┼───────────┼──────────┼──────────┼──────────┼──────────┤
│ 100000     │  1.45001  │ 1.14523  │ 1.11876  │ 1.1108   │ 1.13505  │
├────────────┼───────────┼──────────┼──────────┼──────────┼──────────┤
│      1e+06 │ 11.0856   │ 7.49844  │ 6.26726  │ 5.73358  │ 5.47913  │
╘════════════╧═══════════╧══════════╧══════════╧══════════╧══════════╛


In [36]:
err_np = np.array(err).reshape(4, -1)
table = tabulate(np.hstack([np.array(n).reshape(4, 1), err_np]), num_thr, tablefmt="fancy_grid")

print(table)

╒════════════╤═════════════╤═════════════╤═════════════╤═════════════╤═════════════╕
│            │           1 │           2 │           3 │           4 │           5 │
╞════════════╪═════════════╪═════════════╪═════════════╪═════════════╪═════════════╡
│   1000     │ 2.97544e-12 │ 2.97544e-12 │ 2.97544e-12 │ 2.97544e-12 │ 2.97544e-12 │
├────────────┼─────────────┼─────────────┼─────────────┼─────────────┼─────────────┤
│  10000     │ 5.38838e-10 │ 5.38838e-10 │ 5.38838e-10 │ 5.38838e-10 │ 5.38838e-10 │
├────────────┼─────────────┼─────────────┼─────────────┼─────────────┼─────────────┤
│ 100000     │ 5.29746e-08 │ 5.29746e-08 │ 5.29746e-08 │ 5.29746e-08 │ 5.29746e-08 │
├────────────┼─────────────┼─────────────┼─────────────┼─────────────┼─────────────┤
│      1e+06 │ 3.23341e-06 │ 3.23341e-06 │ 3.23341e-06 │ 3.23341e-06 │ 3.23341e-06 │
╘════════════╧═════════════╧═════════════╧═════════════╧═════════════╧═════════════╛
